In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from datetime import datetime
start_time = datetime.now()
# do your work here

import pandas as pd
import numpy as np
import glob, os 
df=pd.read_csv("/kaggle/input/finaldata09/new0.9.csv")
df




In [ ]:
!rm -rf ./logs/ 
!mkdir ./logs/

In [ ]:
df.isnull().any()

In [ ]:
y=df["prediction"]
y

In [ ]:
x=df.iloc[:,0:170]
x

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
x_min_max_scale=  pd.DataFrame(MinMaxScaler().fit_transform(x))
# Load the TensorBoard notebook extension
%load_ext tensorboard
import tensorflow as tf
import datetime

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_min_max_scale,y , test_size=0.3, random_state=42) # 70% training and 30% test

In [ ]:
x_train=np.expand_dims(x_train, axis=2)
x_test=np.expand_dims(x_test, axis=2)
print(x_test.shape)
print(y_test.shape)
print(x_train.shape)
print(y_train.shape)

In [ ]:
from keras.layers import Input, LSTM, Dense, TimeDistributed, Activation, BatchNormalization, Dropout, Bidirectional
from keras.models import Sequential
from keras.utils import Sequence
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D,Dropout,BatchNormalization

import tensorflow as tf

LR = 0.0009 # maybe after some (10-15) epochs reduce it to 0.0008-0.0007
drop_out = 0.38
batch_dim = 64
nn_epochs = 35

#loss = 'categorical_hinge' # ok
loss = 'categorical_crossentropy' # best standart
#loss = 'mean_absolute_error' # bad
#loss = 'mean_squared_logarithmic_error' # new best (a little better)
def cnn_model2():
    m = Sequential()
    m.add(Conv1D(8, kernel_size = 10, strides = 1, activation='relu', input_shape=(170,1)))
    m.add(BatchNormalization())
    m.add(MaxPooling1D(pool_size=2))
    m.add(Dropout(0.4))
    m.add(Conv1D(16, kernel_size = 10, strides = 1, activation='relu'))
    m.add(BatchNormalization())
    m.add(MaxPooling1D(pool_size=2))
    m.add(Dropout(drop_out))
    m.add(Conv1D(32, kernel_size = 10, strides = 1, activation='relu'))
    m.add(BatchNormalization())
    m.add(MaxPooling1D(pool_size=2))
    m.add(Dropout(0.42))
    m.add(Conv1D(64, kernel_size = 10, strides = 1, activation='relu'))
    m.add(BatchNormalization())
    m.add(MaxPooling1D(pool_size=2))
    m.add(Dropout(0.5))
    m.add(Bidirectional(CuDNNLSTM(128, return_sequences = True, return_state = False)))
    m.add(Bidirectional(CuDNNLSTM(64, return_sequences = True, return_state = False)))
    m.add(Flatten())
    m.add(Dense(256, activation='sigmoid'))
    m.add(Dense(128, activation='sigmoid'))
    m.add(Dense(64, activation='sigmoid'))
    m.add(Dense(32, activation='sigmoid'))
    
    m.add(Dense(2, activation = 'sigmoid'))
    opt = tf.keras.optimizers.Adam(lr=LR)
    m.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=['accuracy'])
   # m.summary()
    return m

In [ ]:
x_train.shape

In [ ]:
def cnn_model3():
    m = Sequential()
    m.add(Conv1D(16, 5, padding='same', activation='sigmoid', input_shape=(170, 1)))
    m.add(BatchNormalization())
    m.add(MaxPooling1D(pool_size=2))
    m.add(Dropout(0.4))
    m.add(Conv1D(16, 3, padding='same', activation='sigmoid'))
    m.add(BatchNormalization())
    m.add(MaxPooling1D(pool_size=2))
    m.add(Dropout(drop_out))
    m.add(Conv1D(32, 3, padding='same', activation='sigmoid'))
    m.add(BatchNormalization())
    m.add(MaxPooling1D(pool_size=2))
    m.add(Dropout(0.42))
    m.add(Conv1D(32, 3, padding='same', activation='sigmoid'))
    m.add(BatchNormalization())
    m.add(MaxPooling1D(pool_size=2))
    m.add(Dropout(0.6))
    #m.add(Conv1D(32, 3, padding='same', activation='relu'))
    #m.add(BatchNormalization())
    #m.add(MaxPooling1D(pool_size=2))
    #m.add(Dropout(0.64))
    m.add(Bidirectional(CuDNNLSTM(128, return_sequences = True, return_state = False)))
    m.add(Bidirectional(CuDNNLSTM(64, return_sequences = True, return_state = False)))
    m.add(Flatten())
    m.add(Dense(256, activation='sigmoid'))
    m.add(Dense(128, activation='sigmoid'))
    m.add(Dense(64, activation='sigmoid'))
    m.add(Dense(32, activation='sigmoid'))
    
    m.add(Dense(2, activation = 'sigmoid'))
    opt = tf.keras.optimizers.Adam(lr=LR)
    m.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=['accuracy'])
   # m.summary()
    return m

In [ ]:
from keras.callbacks import LearningRateScheduler, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint, EarlyStopping

n_folds=5
epochs=1
batch_size=128
pat = 5
#this is the number of epochs with no improvment after which the training will stop
early_stopping = EarlyStopping(monitor='val_loss', patience=5000, min_delta=0, verbose=1 , mode="auto",baseline=None,restore_best_weights=False)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=5000, verbose=1, factor=0.1, min_lr=0.00001)
#define the model checkpoint callback -> this will keep on saving the model as a physical file
model_checkpoint = ModelCheckpoint('subjek1CNN.h5', verbose=1, save_best_only=True)

#define a function to fit the model
def fit_and_evaluate(x_train, x_test, y_train, y_test, EPOCHS=epochs, BATCH_SIZE=batch_size):
    model = None
    model = cnn_model4() 
    results = model.fit(x_train, y_train,epochs=EPOCHS, batch_size=BATCH_SIZE,steps_per_epoch=10000,verbose=1,validation_data=(x_test, y_test),callbacks=[learning_rate_reduction,early_stopping,model_checkpoint], validation_split=0.3)   
    acc=model.evaluate(x_test, y_test)
    print("Val Score: ", model.evaluate(x_test, y_test))
    print("Loss:", acc[0], "Accuracy:", acc[1])
    pred = model.predict(x_test)
    pred_y = pred.argmax(axis=-1)
    pred_y
    accuracy = accuracy_score(y_test, pred_y)
    print(accuracy)
    return results,pred_y,pred


In [ ]:
history,pred_y= fit_and_evaluate(x_train,x_test,y_train,y_test,150,32)

In [ ]:
model=cnn_model4()

In [ ]:
history_dict = history.history
print(history_dict.keys())

In [ ]:
import matplotlib.pyplot as plt 
from sklearn import metrics 
plt.figure(dpi=1500)
import matplotlib.pyplot as pyplot
pyplot.title('Accuracy')
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.show()
plt.savefig("accuracy.png",dpi=1500)

In [ ]:
plt.figure(dpi=1280)

pre, rec, thr = metrics.precision_recall_curve(y_test, prob_test)
plt.figure(figsize=(8,4))
plt.plot(thr, pre[:-1], label='precision')
plt.plot(thr, rec[1:], label='recall')
plt.xlabel('Threshold')
plt.title('Precision & Recall vs Threshold', c='r', size=16)
plt.legend()
plt.savefig('Precision&RecallvsThreshold.png', dpi=1500)

plt.show() 

In [ ]:
len(loss_train)

In [ ]:
n_folds=3
#save the model history in a list after fitting so that we can plot later
model_history = [] 

for i in range(n_folds):
    print("Training on Fold: ",i+1)
    t_x, val_x, t_y, val_y = train_test_split(x_train, y_train,test_size=0.3, random_state = np.random.randint(1,250,8)[0])
    model_history.append(fit_and_evaluate(x_train, x_test, y_train, y_test, epochs, batch_size))
   
    print("======="*12, end="\n\n\n")

In [ ]:
%tensorboard --logdir logs/fit
